In [1]:
import os

import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.gaia import Gaia
from astroquery.simbad import Simbad

variableName = os.getcwd().split('/')[-1]
variableName

'LINEAR_44914'

# Configure dustmaps if not done already

In [2]:
import dustmaps.bayestar as bayestar

bayestar.fetch() # fetch dustmap if not already retrieved

Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.


# Retrieve GDR3 Object Information

Use SIMBAD to map object name to matched GDR3 `source_id`.

Retrieve photogeometric distance from Bailer-Jones et al. (2021) table `external.gaiaedr3_distance`.

In [3]:
simbadResult = Simbad.query_objectids(variableName)
allIds = simbadResult['ID']
gaiaSourceId = [int(s_id.replace('Gaia DR3 ', '')) for s_id in allIds if 'Gaia DR3' in s_id][0]
gaiaSourceId, type(gaiaSourceId)

(934267300844153600, int)

Flags for table results: ABBCC

Taken from Bailer-Jones et al. (2021)

A Source magnitude compared to the limit used to make the prior
```
        0 Source has no G-band magnitude
        1 G <= Glim
        2 G > Glim
```

B Hartigan dip test for unimodality. Left digit, geometric; right digit, photogeometric
```
        0 Unimodal hypothesis okay
        1 Possible evidence for multimodality
```

C Q<sub>G</sub> models used in prior. Left digit, bluer model; right digit, redder model
```
        0 Null (no model)
        1 One-component Gaussian
        2 Two-component Gaussian
        3 Smoothing spline
        Special setting:
        99 Source lacks G and/or BP − RP
```

In [4]:
query = f"""
        select *
        from external.gaiaedr3_distance
        where source_id = '{gaiaSourceId}'
        """
job = Gaia.launch_job(query)
results = job.get_results()
results.to_pandas()

,source_id,r_med_geo,r_lo_geo,r_hi_geo,r_med_photogeo,r_lo_photogeo,r_hi_photogeo,flag
0,934267300844153600,510.735748,502.984253,519.545837,508.533447,502.359131,516.09613,10033


In [5]:
# can take median photogeometric distance by default, but this way we can check the distance systematically for each measurement
distanceCols = results.colnames.copy()
distanceCols.remove('source_id')
distanceCols.remove('flag')
distanceCols

['r_med_geo',
 'r_lo_geo',
 'r_hi_geo',
 'r_med_photogeo',
 'r_lo_photogeo',
 'r_hi_photogeo']

# Calculate $E(B-V)$ using Bailer-Jones et al. (2021) distance

In [6]:
# for compute time's sake, only check median photogeometric distance for now
distance = results['r_med_photogeo'][0]*u.pc # parsecs units from Gaia table
distance

<Quantity 508.53345 pc>

In [7]:
variableCoord = SkyCoord.from_name(variableName)
variableCoord = SkyCoord(variableCoord.ra, variableCoord.dec, distance=distance)
variableCoord

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (118.06521155, 49.79953783, 508.53345)>

In [8]:
dustmap = bayestar.BayestarQuery(max_samples=100)
ebv, flags = dustmap(variableCoord, mode='samples', return_flags=True)
ebv, flags # might end up having some appreciable effect

Loading pixel_info ...
Loading samples ...
Loading best_fit ...
Replacing NaNs in reliable distance estimates ...
Sorting pixel_info ...
Extracting hp_idx_sorted and data_idx at each nside ...
  nside = 64
  nside = 128
  nside = 256
  nside = 512
  nside = 1024
t = 23.639 s
  pix_info:   0.375 s
   samples:  13.608 s
      best:   2.893 s
       nan:   0.018 s
      sort:   6.685 s
       idx:   0.061 s


(array([0.09999999, 0.11      , 0.09999999, 0.09      , 0.02769466],
       dtype=float32),
 (True, True))